In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.entry_processed as entry_processed
import xfmkit.entry_raw as entry_raw
import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.config as config

#add the parent folder to sys path so we can import from within the notebook subfolder
#sys.path.insert(0,'..')

print(sys.path)

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full -n 2 -fc'
args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/nf_demo_full", "-som", "-ff"]
#args_in = [ "-f", "/home/lachlan/CODEBASE/xfmkit/data/example_datafile.GeoPIXE", "-a", "-m", "-dt", "-1" ]

PACKAGE_CONFIG='xfmkit/config.yaml'


In [ ]:
#pxs, xfmap, = entry_raw.read_raw(args_in)
pxs, embedding, categories, classavg, palette, kde = entry_processed.read_processed(args_in)

In [ ]:
import intrasom

# Results Clustering and Plotting Modules
from intrasom.visualization import PlotFactory
from intrasom.clustering import ClusterFactory


In [ ]:
data = pxs.data.d

mapsize = (24,14)
som_test = intrasom.SOMFactory.build(data,
                                     mapsize=mapsize,
                                     mapshape='toroid',
                                     lattice='hexa',
                                     normalization='var',
                                     initialization='random',
                                     neighborhood='gaussian',
                                     training='batch',
                                     name='Example',
                                     component_names=None,
                                     unit_names = None,
                                     sample_names=None,
                                     missing=False,
                                     pred_size=0)

In [ ]:
som_test.train(train_len_factor=2,
               previous_epoch = True)

In [ ]:
plot = PlotFactory(som_test)

plot.plot_umatrix(figsize = (13,2.5),
                  hits = False,
                  title = "U-Matrix",
                  title_size = 20,
                  title_pad = 20,
                  legend_title = "Distance",
                  legend_title_size = 12,
                  legend_ticks_size = 7,
                  label_title_xy = (0,0.5),
                  save = False,
                  watermark_neurons=False)

In [ ]:
clustering = ClusterFactory(som_test)
clusters = clustering.kmeans(k=10)

In [ ]:
clustering.plot_kmeans(figsize = (12,5),
                       clusters = clusters,
                       title_size = 18,
                       title_pad = 20,
                       umatrix=True,
                       colormap = "gist_rainbow",
                       alfa_clust=0.5,
                       hits=True,
                       legend_text_size =7,
                       cluster_outline=False,
                       save=False)

In [ ]:
clustering.results_cluster(clusters)